In [5]:
import os
import sys

# os.chdir('../')
print(os.getcwd())
sys.path.append(os.getcwd())

In [2]:
import pandas as pd
from tqdm import trange, tqdm
from multiprocessing import Pool, Manager
from scipy.stats import fisher_exact
import numpy as np
import networkx as nx

In [6]:
from utils.clustering_utils import seqs2hamming, get_epitopes_for_cluster, check_distance

In [7]:
alpha_cluster = 29
beta_cluster = 2

clean_beta_cm = pd.read_csv('data/significant_clone_matrix_fisher_fmba_TRB_top_500k_wo_leaks.csv').drop(
    columns=['Unnamed: 0'])
covid_clones_beta = clean_beta_cm.columns[1:]
res_beta = seqs2hamming(covid_clones_beta, viz_method='drl')

clean_alpha_cm = pd.read_csv('data/significant_clone_matrix_fisher_fmba_TRA_top_500k_wo_leaks.csv').drop(
    columns=['Unnamed: 0'])
covid_clones_alpha = clean_alpha_cm.columns[1:]
res_alpha = seqs2hamming(covid_clones_alpha, viz_method='drl')

vdjdb = pd.read_csv('data/vdjdb.txt', sep='\t')
ab_vdjdb = pd.read_csv('data/vdjdb_full.txt', sep='\t')

alpha_epi = set(get_epitopes_for_cluster(vdjdb, res_alpha, alpha_cluster, dist=1)['antigen.epitope'])
beta_epi = set(get_epitopes_for_cluster(vdjdb, res_beta, beta_cluster, dist=1)['antigen.epitope'])

intersected_epi = alpha_epi.intersection(beta_epi)


/scratch/ipykernel_1502080/1783781562.py:15: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  ab_vdjdb = pd.read_csv('data/vdjdb_full.txt', sep='\t')


In [8]:
vdjdb

,complex.id,gene,cdr3,v.segm,j.segm,species,mhc.a,mhc.b,mhc.class,antigen.epitope,...,antigen.species,reference.id,method,meta,cdr3fix,vdjdb.score,web.method,web.method.seq,web.cdr3fix.nc,web.cdr3fix.unmp
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...",2,sort,sanger,no,no
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2,sort,sanger,no,no
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAVPSGAGSYQLTF"", ""cdr3_old"": ""CAVPSG...",2,sort,sanger,no,no
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,...,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2,sort,sanger,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92766,30592,TRB,CASSPGQGGDNEQFF,TRBV7-3*01,TRBJ2-1*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,...,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSPGQGGDNEQFF"", ""cdr3_old"": ""CASSP...",0,sort,singlecell,no,no
92767,30593,TRA,CAPQGATNKLIF,TRAV12-2*01,TRAJ32*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,...,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAPQGATNKLIF"", ""cdr3_old"": ""CAPQGATN...",2,sort,singlecell,no,no
92768,30593,TRB,CASSLGAGGQETQYF,TRBV5-1*01,TRBJ2-5*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,...,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSLGAGGQETQYF"", ""cdr3_old"": ""CASSL...",2,sort,singlecell,no,no
92769,30594,TRA,CLVGGSGGYNKLIF,TRAV4*01,TRAJ4*01,HomoSapiens,HLA-DQA1*05:01,HLA-DQB1*02:01,MHCII,PQQPFPQPEQPFP,...,Wheat,PMID:33927715,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD4+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CLVGGSGGYNKLIF"", ""cdr3_old"": ""CLVGGS...",0,sort,singlecell,no,no


In [9]:
intersected_epi

{'KLGGALQAK'}

In [10]:
res_alpha

,cdr3,cluster,x,y,cluster_size,x_mean,y_mean
0,CAAADFNKFYF,0,212.942352,-112.093819,460,176.681889,-171.258318
1,CAAANFNKFYF,0,173.135300,-200.564957,460,176.681889,-171.258318
2,CAADDFNKFYF,0,212.143631,-112.240715,460,176.681889,-171.258318
3,CAAGDFNKFYF,0,212.068985,-111.738663,460,176.681889,-171.258318
4,CAAKDFNKFYF,0,211.578659,-112.445190,460,176.681889,-171.258318
...,...,...,...,...,...,...,...
4392,CVVSDLMEYGNKLVF,111,-52.155739,-346.616791,2,-52.243322,-346.867569
4393,CVVSDRMEYGNKLVF,111,-52.330906,-347.118347,2,-52.243322,-346.867569
4394,CVVSEPSGGYQKVTF,112,13.418700,-350.782745,1,13.418700,-350.782745
4395,CVVSGYGAAGNKLTF,113,255.128906,-232.216873,1,255.128906,-232.216873


In [11]:
res_beta

,cdr3,cluster,x,y,cluster_size,x_mean,y_mean
0,CASAPGGSYEQYF,0,-83.934418,-56.398754,201,0.074998,-76.001667
1,CASIPGGSYEQYF,0,-84.065620,-55.987762,201,0.074998,-76.001667
2,CASKPGGSYEQYF,0,-83.843422,-56.817844,201,0.074998,-76.001667
3,CASLPGGSYEQYF,0,-83.485710,-56.176773,201,0.074998,-76.001667
4,CASNPGGSYEQYF,0,-84.283310,-56.544952,201,0.074998,-76.001667
...,...,...,...,...,...,...,...
563,CSASSGHEQYF,57,147.564438,-20.576210,3,147.356216,-20.729224
564,CSATSGHEQYF,57,147.137527,-20.555742,3,147.356216,-20.729224
565,CSARGSSYEQYF,58,-331.370087,300.947968,1,-331.370087,300.947968
566,CSASDRGYEQYF,59,-73.226593,259.942169,1,-73.226593,259.942169


In [12]:
ab_vdjdb = ab_vdjdb[(~ab_vdjdb['cdr3.alpha'].isna()) & (~ab_vdjdb['cdr3.beta'].isna())].reset_index(drop=True)

In [13]:
ab_vdjdb.head()

,cdr3.alpha,v.alpha,j.alpha,cdr3.beta,v.beta,d.beta,j.beta,species,mhc.a,mhc.b,...,meta.replica.id,meta.clone.id,meta.epitope.id,meta.tissue,meta.donor.MHC,meta.donor.MHC.method,meta.structure.id,cdr3fix.alpha,cdr3fix.beta,vdjdb.score
0,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,NaN,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,...,NaN,NaN,NaN,PBMC,"HLA-A*02:01,HLA-A*24:02;HLA-B*08:01,HLA-B*5701...",NaN,NaN,"{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
1,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,CASSFEPGQGFYSNQPQHF,TRBV13*01,NaN,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,...,NaN,NaN,NaN,PBMC,"HLA-A*02,HLA-A*24:02;HLA-B*08:01,HLA-B*15;HLA-...",NaN,NaN,"{""cdr3"": ""CAVPSGAGSYQLTF"", ""cdr3_old"": ""CAVPSG...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2
2,CAVKASGSRLT,TRAV2*01,NaN,CASSYEPGQVSHYSNQPQHF,TRBV13*01,NaN,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,...,NaN,NaN,NaN,PBMC,"HLA-A*01:01,HLA-A*02:01;HLA-B*08:01,HLA-B*57:0...",NaN,NaN,"{""cdr3"": ""CAVKASGSRLT"", ""cdr3_old"": ""CAVKASGSR...","{""cdr3"": ""CASSYEPGQVSHYSNQPQHF"", ""cdr3_old"": ""...",2
3,CAYRPPGTYKYIF,TRAV38-2/DV8*01,TRAJ40*01,CASSALASLNEQFF,TRBV14*01,NaN,TRBJ2-1*01,HomoSapiens,HLA-B*08,B2M,...,NaN,NaN,NaN,PBMC,"HLA-A*01:01,HLA-A*02:01;HLA-B*08:01,HLA-B*57:0...",NaN,NaN,"{""cdr3"": ""CAYRPPGTYKYIF"", ""cdr3_old"": ""CAYRPPG...","{""cdr3"": ""CASSALASLNEQFF"", ""cdr3_old"": ""CASSAL...",2
4,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,CASSYLPGQGDHYSNQPQHF,TRBV13*01,NaN,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,...,NaN,NaN,NaN,TCL culture,"HLA-A*02:01,HLA-A*24:02;HLA-B*08:01,HLA-B*5701...",NaN,NaN,"{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2


In [14]:
res_alpha[res_alpha.cluster == alpha_cluster].head()

,cdr3,cluster,x,y,cluster_size,x_mean,y_mean
3972,CAFMKPQGGSEKLVF,29,-39.705803,375.311401,1,-39.705803,375.311401


In [15]:
res_beta[res_beta.cluster == beta_cluster].head()

,cdr3,cluster,x,y,cluster_size,x_mean,y_mean
203,CASGQGGYEQYF,2,-53.055904,37.248753,112,-0.227757,37.540469
204,CASSQGGYEQYF,2,-53.235054,37.622154,112,-0.227757,37.540469
205,CASSADSYEQYF,2,87.821732,41.150265,112,-0.227757,37.540469
206,CASSATSYEQYF,2,79.224419,36.639591,112,-0.227757,37.540469
207,CASSDDSYEQYF,2,86.637039,41.183449,112,-0.227757,37.540469


In [16]:
def check_db_epitopes_cdr3(db, cdr3, cdr3_column, dist=1):
    return db[db[cdr3_column].apply(lambda x: check_distance(x, cdr3, dist))]

In [17]:
check_db_epitopes_cdr3(ab_vdjdb[ab_vdjdb['antigen.epitope'] == 'ELAGIGILTV'], 'CASGLGGNQPQHF', 'cdr3.beta')

,cdr3.alpha,v.alpha,j.alpha,cdr3.beta,v.beta,d.beta,j.beta,species,mhc.a,mhc.b,...,meta.replica.id,meta.clone.id,meta.epitope.id,meta.tissue,meta.donor.MHC,meta.donor.MHC.method,meta.structure.id,cdr3fix.alpha,cdr3fix.beta,vdjdb.score


In [18]:
alpha_cluster_to_epi = {x: set(get_epitopes_for_cluster(vdjdb, 
                                                        res_alpha, 
                                                        x, 
                                                        dist=1)['antigen.epitope']) for x in range(res_alpha.cluster.max() + 1)}

In [19]:
beta_cluster_to_epi = {x: set(get_epitopes_for_cluster(vdjdb, 
                                                        res_beta, 
                                                        x, 
                                                        dist=1)['antigen.epitope']) for x in range(res_beta.cluster.max() + 1)}

In [20]:
def process_alpha_beta_pair(args):
    alpha_cluster, beta_cluster = args
    alpha_epi = alpha_cluster_to_epi[alpha_cluster]
    beta_epi = beta_cluster_to_epi[beta_cluster]

    intersected_epi = alpha_epi.intersection(beta_epi)

    for epi in intersected_epi:
        edge_to_vdjdb_instance = {}
        G = nx.Graph()
        for alpha_index, alpha_clone in enumerate(res_alpha[res_alpha.cluster == alpha_cluster].cdr3):
            db_with_alpha_clone = check_db_epitopes_cdr3(
                ab_vdjdb[ab_vdjdb['antigen.epitope'] == epi], 
                alpha_clone, 
                'cdr3.alpha')
            if len(db_with_alpha_clone) != 0:
                for beta_index, beta_clone in enumerate(res_beta[res_beta.cluster == beta_cluster].cdr3):
                    db_with_alpha_and_beta = check_db_epitopes_cdr3(
                        db_with_alpha_clone,
                        beta_clone,
                        'cdr3.beta')
                    if len(db_with_alpha_and_beta)!= 0:
                        edge_to_vdjdb_instance[alpha_index, beta_index] = db_with_alpha_and_beta[['cdr3.alpha', 'cdr3.beta', 
                                                                                                  'antigen.epitope', 'antigen.species']]
                        G.add_edge('a_'+str(alpha_index), 'b_'+str(beta_index), weight=len(db_with_alpha_and_beta))
        max_matching = nx.max_weight_matching(G, maxcardinality=True)
        if len(max_matching) != 0:
            print(alpha_cluster, beta_cluster)
            pairs = []
            for pair in max_matching:
                if pair[0].startswith('b'):
                    pair = pair[1], pair[0]
                pairs.append((int(pair[0].split('_')[1]), int(pair[1].split('_')[1])))
            res = ''
            resulting_dataset = pd.concat([edge_to_vdjdb_instance[pair] for pair in pairs])
            
            min_of_cluster_sizes = min(len(res_alpha[res_alpha.cluster == alpha_cluster].cdr3), 
                                       len(res_beta[res_beta.cluster == beta_cluster].cdr3))
            lines_in_vdjdb_with_epi = len(ab_vdjdb[ab_vdjdb['antigen.epitope'] == epi])
                        
            pval = fisher_exact([[lines_in_vdjdb_with_epi, len(ab_vdjdb) - lines_in_vdjdb_with_epi],
                                 [len(resulting_dataset), max(0, min_of_cluster_sizes - len(resulting_dataset))]], alternative='less')[1]

            if len(resulting_dataset) != 0:
                res += f'{epi} {alpha_cluster} {beta_cluster} {pairs} {pval}\n'
                res += f'{resulting_dataset}\n'
                print(res)
            resulting_list.append([alpha_cluster, beta_cluster, epi, pval, len(resulting_dataset), resulting_dataset.drop_duplicates()])

In [21]:
resulting_list = Manager().list()
with Pool(30) as p:
    p.map(process_alpha_beta_pair, [(x, y) for x in range(res_alpha.cluster.max() + 1) for y in range(res_beta.cluster.max() + 1)])

34 52
TFEYVSQPFLMDLE 34 52 [(47, 0)] 0.00018177416482371064
            cdr3.alpha      cdr3.beta antigen.epitope antigen.species
30171  CAALNYGGSQGNLIF  CASSYRGAYGYTF  TFEYVSQPFLMDLE      SARS-CoV-2
30523  CAGLNYGGSQGNLIF  CASSSRGAFGYTF  TFEYVSQPFLMDLE      SARS-CoV-2

44 0
GILGFVFTL 44 0 [(15, 10)] 0.8927213253245376
         cdr3.alpha      cdr3.beta antigen.epitope antigen.species
7329  CAASASGTYKYIF  CASSARSSYEQYF       GILGFVFTL      InfluenzaA

46 0
KLGGALQAK 46 0 [(4, 80)] 0.9999999293922825
           cdr3.alpha      cdr3.beta antigen.epitope antigen.species
11620  CAEIHTSGTYKYIF  CASSLTGGYEQYF       KLGGALQAK             CMV

46 0
AVFDRKSDAK 46 0 [(13, 155)] 0.7988371227717442
           cdr3.alpha      cdr3.beta antigen.epitope antigen.species
20432  CAASATSGTYKYIF  CASSLEPSYEQYF      AVFDRKSDAK             EBV

46 2
KLGGALQAK 46 2 [(13, 77)] 0.9999999293922825
           cdr3.alpha     cdr3.beta antigen.epitope antigen.species
22613  CAASATSGTYKYIF  CASSLLGNEQYF       KLGGA

In [22]:
alpha_clusters, beta_clusters, epitopes, pvals, number_of_matched_pairs, beta_cdr3s, alpha_cdr3s = [], [], [], [], [], [], []
for a, b, epi, pval, num_pairs, df in resulting_list:
    cdr3_a, cdr3_b = df.reset_index()['cdr3.alpha'][0], df.reset_index()['cdr3.beta'][0]
    alpha_clusters.append(a)
    beta_clusters.append(b)
    epitopes.append(epi)
    pvals.append(pval)
    number_of_matched_pairs.append(num_pairs)
    beta_cdr3s.append(cdr3_b)
    alpha_cdr3s.append(cdr3_a)
res_df = pd.DataFrame(data={'alpha_cluster': alpha_clusters,
                    'beta_cluster': beta_clusters,
                    'antigen.epitope': epitopes,
                    'pval': pvals,
                    'num_matched_pairs': number_of_matched_pairs,
                    'cdr3.beta': beta_cdr3s,
                    'cdr3.alpha': alpha_cdr3s}).merge(vdjdb[['antigen.epitope', 'antigen.species']]).drop_duplicates()

In [23]:
res_df

,alpha_cluster,beta_cluster,antigen.epitope,pval,num_matched_pairs,cdr3.beta,cdr3.alpha,antigen.species
0,34,52,TFEYVSQPFLMDLE,0.000182,2,CASSYRGAYGYTF,CAALNYGGSQGNLIF,SARS-CoV-2
937,44,0,GILGFVFTL,0.892721,1,CASSARSSYEQYF,CAASASGTYKYIF,InfluenzaA
10961,25,0,GILGFVFTL,0.823785,2,CASSSRSSYEQYF,CAVDNTDKLIF,InfluenzaA
20985,34,0,GILGFVFTL,0.203956,7,CASSTRSSYEQYF,CAGQLGGGSQGNLIF,InfluenzaA
31009,14,0,GILGFVFTL,0.200153,1,CASSARSSYEQYF,CAASASGTYKYIF,InfluenzaA
41033,4,0,GILGFVFTL,1.000000,1,CASSARSSYEQYF,CAVRNQGGSEKLVF,InfluenzaA
51057,0,0,GILGFVFTL,1.000000,1,CASSIRSSYEQYF,CAPYNFNKFYF,InfluenzaA
61081,1,0,GILGFVFTL,1.000000,1,CASSIRSSYEQYF,CAVGDNFNKFYF,InfluenzaA
71105,2,0,GILGFVFTL,1.000000,1,CASSIRSSYEQYF,CALSEGNFNKFYF,InfluenzaA
81129,46,0,KLGGALQAK,1.000000,1,CASSLTGGYEQYF,CAEIHTSGTYKYIF,CMV


In [24]:
res_df[res_df.pval < 0.05]

,alpha_cluster,beta_cluster,antigen.epitope,pval,num_matched_pairs,cdr3.beta,cdr3.alpha,antigen.species
0,34,52,TFEYVSQPFLMDLE,0.000182,2,CASSYRGAYGYTF,CAALNYGGSQGNLIF,SARS-CoV-2
308259,98,0,TTDPSFLGRY,0.013074,1,CASSPGTTYEQYF,CAVRAYNTDKLIF,SARS-CoV-2
309106,100,0,YLQPRTFLL,0.015460,1,CASSLYGGYEQYF,CAVNTGFQKLVF,SARS-CoV-2
313122,98,3,RLRAEAQVK,0.014120,1,CASSLFHEQYF,CAVRLYNTDKLIF,EBV
313984,100,3,ELAGIGILTV,0.013205,1,CASSNSYEQYF,CAVDTGFQKLVF,HomoSapiens
316671,84,3,ELAGIGILTV,0.013205,1,CASSNSYEQYF,CAVDTGFQKLVF,HomoSapiens
319358,84,2,IPVAYRKVL,0.000163,1,CASSLPGWEQYF,CATDTGFQKLVF,SARS-CoV-2
322122,4,5,FLRGRAYGL,0.044285,1,CASSLGGGDYEQYF,CAASAQGGSEKLVF,EBV
322243,4,15,FLGIITTVA,0.000294,1,CASSLGGEGYEQYF,CALNLQGGSEKLVF,SARS-CoV-2


In [25]:
sign_res_df = res_df[res_df.pval < 0.05]

In [26]:
annot_df = np.zeros((res_alpha.cluster.max()+1, res_beta.cluster.max() + 1))
for a, b in zip(sign_res_df.alpha_cluster, sign_res_df.beta_cluster):
    annot_df[a, b] = 1

In [27]:
pd.DataFrame(annot_df).to_csv('data/alpha_beta_paired_epitopes_by_full_db.csv', index=False)

In [28]:
corr = pd.read_csv('data/cluster_correlations.csv').to_numpy()
corr

array([[0.3031276 , 0.11983096, 0.32706148, ..., 0.1384917 , 0.07326477,
        0.13223421],
       [0.31792309, 0.11274129, 0.33746652, ..., 0.1167759 , 0.10299098,
        0.11487081],
       [0.30709934, 0.116705  , 0.33352355, ..., 0.11995352, 0.10256326,
        0.12503543],
       ...,
       [0.22947409, 0.10687781, 0.20513775, ..., 0.0721069 , 0.06527039,
        0.04473625],
       [0.09026371, 0.01140255, 0.07618391, ..., 0.09946874, 0.04088858,
        0.00147463],
       [0.32662167, 0.09250165, 0.31164981, ..., 0.16202533, 0.09053542,
        0.13719387]])

In [29]:
sign_res_df['corr_coeff'] = sign_res_df.apply(lambda x: corr[x.alpha_cluster][x.beta_cluster], axis=1)

/scratch/ipykernel_1502080/907165014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_res_df['corr_coeff'] = sign_res_df.apply(lambda x: corr[x.alpha_cluster][x.beta_cluster], axis=1)


In [31]:
corr = pd.read_csv('data/cluster_odds_ratio.csv').to_numpy()
corr

array([[1.61004106e-03, 1.89574774e-03, 1.63835510e-03, ...,
        1.91519564e-03, 1.78153094e-03, 2.06024354e-03],
       [8.86832633e-04, 9.67393430e-04, 9.02366959e-04, ...,
        9.83440744e-04, 1.06376343e-03, 1.02791989e-03],
       [9.06710776e-04, 1.01879601e-03, 9.19605419e-04, ...,
        1.01691617e-03, 1.09550154e-03, 1.07360402e-03],
       ...,
       [3.32878255e-03, 6.96105062e-01, 4.80114851e-03, ...,
        8.06313045e-01, 9.50847458e-01, 8.08474576e-01],
       [3.32878255e-03, 5.30747398e-01, 4.80114851e-03, ...,
        1.19462117e+00, 1.13839286e+00, 7.23214286e-01],
       [3.32878255e-03, 4.99368181e-01, 4.80114851e-03, ...,
        6.46042217e-01, 6.68706294e-01, 7.01298701e-01]])

In [32]:
sign_res_df['odds'] = sign_res_df.apply(lambda x: corr[x.alpha_cluster][x.beta_cluster], axis=1)

/scratch/ipykernel_2645489/3198760451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sign_res_df['odds'] = sign_res_df.apply(lambda x: corr[x.alpha_cluster][x.beta_cluster], axis=1)


In [33]:
sign_res_df

,alpha_cluster,beta_cluster,antigen.epitope,pval,num_matched_pairs,cdr3.beta,cdr3.alpha,antigen.species,corr_coeff,odds_ratio,odds
0,34,52,TFEYVSQPFLMDLE,0.000182,2,CASSYRGAYGYTF,CAALNYGGSQGNLIF,SARS-CoV-2,0.161858,0.004899,0.004899
311015,98,0,TTDPSFLGRY,0.013074,1,CASSPGTTYEQYF,CAVRAYNTDKLIF,SARS-CoV-2,0.489740,0.003329,0.003329
311862,100,0,YLQPRTFLL,0.015460,1,CASSLYGGYEQYF,CAVNTGFQKLVF,SARS-CoV-2,0.475245,0.003329,0.003329
315878,98,3,RLRAEAQVK,0.014120,1,CASSLFHEQYF,CAVRLYNTDKLIF,EBV,0.469545,0.003154,0.003154
316740,100,3,ELAGIGILTV,0.013205,1,CASSNSYEQYF,CAVDTGFQKLVF,HomoSapiens,0.454781,0.003157,0.003157
319427,84,3,ELAGIGILTV,0.013205,1,CASSNSYEQYF,CAVDTGFQKLVF,HomoSapiens,0.268994,0.003157,0.003157
322114,84,2,IPVAYRKVL,0.000163,1,CASSLPGWEQYF,CATDTGFQKLVF,SARS-CoV-2,0.283979,0.004801,0.004801
322122,4,5,FLRGRAYGL,0.044285,1,CASSLGGGDYEQYF,CAASAQGGSEKLVF,EBV,0.556690,0.002397,0.002397
322243,4,15,FLGIITTVA,0.000294,1,CASSLGGEGYEQYF,CALNLQGGSEKLVF,SARS-CoV-2,0.414668,0.002440,0.002440


In [31]:
sign_res_df.to_csv('figures/significant_alpha_beta_metaclone_matching.csv')